<a href="https://colab.research.google.com/github/Aribabo/Matchuri/blob/main/GPT2_%ED%95%99%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face의 Trainer 사용한 코드
* 사용된 토크나이져: GPT2TokenizerFast
* 사용된 모델: GPT2LMHeadModel

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate

# 설치해도 안되면 이 코드 주석처리하고 세션 다시시작

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
# 라이브러리 import 부분
from transformers import GPT2TokenizerFast, DataCollatorWithPadding, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import re
import numpy as np
import pandas as pd
import torch
from datasets import DatasetDict, Dataset
from datetime import datetime
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import evaluate

# 고정 변수

In [ ]:
# 현재 시간 얻기
now = datetime.now()
date = now.strftime("%Y-%m-%d")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
date

'2024-02-13'

# 직접 설정해야하는 부분

In [ ]:
# 본인이 가지고 있는 데이터파일 경로로 변경할것
DATA_PATH = '/content/drive/MyDrive/KDT-챗봇 프로젝트/모델실험실/데이터통합본 - 재정제중.xlsx'

# 최종 모델 저장할 경로 설정
# MODEL_SAVE_PATH = f'{"/content/drive/MyDrive/KDT-챗봇 프로젝트/모델실험실/정흔/skt-kogpt2-base-v2"}-{date}-{device}.pth'

# 데이터 train, validation 비율 설정
train_valid_rate = 0.2
RANDOM_STATE = 2024

# max_len 설정
MAX_LEN = 200

# 하이퍼 파라미터 설정
EPOCHS = 20
LR = 5e-5
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
WEIGHT_DECAY = 0.01

In [ ]:
# 토큰 정의
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'
checkpoint = "skt/kogpt2-base-v2" # 지금 데이터로 성능 확인됨 -> 데이터 늘려서 다시 확인 필요

# 모델, 토크나이저 정의
koGPT2_TOKENIZER = GPT2TokenizerFast.from_pretrained(checkpoint,
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)

model = GPT2LMHeadModel.from_pretrained(checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class QADataFrame():
    def __init__(self, path):
        self.path = path
        self.excel_file = pd.ExcelFile(path)
        self.sheet_names = self.excel_file.sheet_names

    def sheetToDataframe(self, sheet_num):
        return pd.read_excel(self.excel_file, self.sheet_names[sheet_num])

    def splitTrainVal(self, df, test_size=0.2, random_state=42):
        # StratifiedShuffleSplit 초기화
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)

        # 레이블 데이터
        labels = df['labels'].values

        # 데이터 분할
        for train_index, valid_index in sss.split(df, labels):
            train_df = df.iloc[train_index]
            valid_df = df.iloc[valid_index]

        return train_df, valid_df

In [ ]:
qa = QADataFrame(DATA_PATH)

# 문제와 정답 데이터프레임
df = qa.sheetToDataframe(0)
sheet_data = qa.sheetToDataframe(1)
sheet_data.dropna(inplace=True)
sheet_data['문제번호'] = sheet_data['문제번호'].astype(int)
new_df = pd.merge(df, sheet_data, on='문제번호', how='outer')
new_df['Q'] = '정답 = ' + new_df['정답'].astype(str) + '질문 = ' + new_df['질문(사용자)'].astype(str)
new_df = new_df.loc[:,['문제번호','Q','답변(챗봇)']]
new_df.columns = ['labels', 'Q', 'A']

new_df = new_df[new_df['labels'].isin([1, 11, 28,22,20,29])]
new_df['labels'] = new_df['labels'].apply(lambda x: 1 if x == 1 else (2 if x == 11 else (3 if x == 28 else (4 if x == 22 else (5 if x == 20 else (6 if x == 29 else x))))))
new_df

,labels,Q,A
0,1,정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 ...,그는 홀로 일을 처리하려 내려가고있었지.
1,1,정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 ...,그는 휴가를 냈지.
2,1,정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 ...,특별한 날은 아니지만 하루하루가 중요한 날은 맞네.
3,1,정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 ...,맞아 매우 위독한 병이야.
4,1,정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네. 상사의 연락을 받고 ...,맞네. 그것이 없다면 아내는 큰일이 날것일세.
...,...,...,...
8404,6,정답 = 이 여학생은 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이...,아니네. 다시 생각해보게
8405,6,정답 = 이 여학생은 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이...,아마 그랬을 것이네
8406,6,정답 = 이 여학생은 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이...,그건 중요하지 않네
8407,6,정답 = 이 여학생은 외국어 구두시험을 앞두고 멀쩡한 팔에 일부로 깁스를 했네. 이...,맞네


In [ ]:
# 사용자 정의 토큰 추가
custom_tokens = []
for i in pd.unique(new_df['labels']):
    txt='<문제' + str(int(i)) + '>'
    custom_tokens.append(txt)

# 토크나이저에 문제 라벨 토큰 추가
koGPT2_TOKENIZER.add_special_tokens({'additional_special_tokens': custom_tokens})

# 추가하고 크기 늘려줘야 에러 안 남
model.resize_token_embeddings(len(koGPT2_TOKENIZER))
print(custom_tokens)

['<문제1>', '<문제2>', '<문제5>', '<문제4>', '<문제3>', '<문제6>']


In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=120):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def get_problem_number_token(self, label):
        return '<문제'+str(label)+'>'

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        # 문제번호 토큰 추가 (이 부분이 수정된 부분)
        problem_number_label = turn["labels"]  # label_column_name에 실제 DataFrame의 레이블 컬럼 이름을 적어주세요
        problem_number_token = self.get_problem_number_token(problem_number_label)  # 해당 레이블에 대한 문제 번호 토큰 선택

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token + problem_number_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        # print('example: ', self.q_token + q + self.sent_token + problem_number_token)
        # print('example:', self.a_token + a + self.eos)

        '''
        example of q_toked:
            <usr>정답 = 말콤은 쉬는 날 아픈 아내의 병원에 간병을 와있었네
            상사의 연락을 받고 계단을 내려가던 중 병원의 비상전력 등 모든 전기가 꺼져버리게 되고 정전이 돼버렸네
            이때 말콤은 위독한 병에 걸려 생명유지장치에 겨우 의지해 살아가던 자신의 아내의 죽음을 확신하고 고통스러워한 것이었네
            질문 = 아내는 감전사 했나요<unused1><문제1>

        example of a_toked:
            <sys>아니네  아내는 살해당하지 않았네</s>

        특이사항:
            - 특수문자와 구두점이 있던 자리에 공백이 추가됨. 즉, 현재 문장 구분이 공백x2임
            - 문장을 구분하는 토큰을 추가해야할지?
            - 정답 문장을 키워드 위주로 줄일 필요가 있을 것 같음, 길이때문에 출력 문장이 짤리는 경우가 있음
            - 정답과 질문에 가중치를 조절할 수 있다면 새로운 결과를 찾을 수 있을 것 같음

        학습에 사용되는 입력 문장은 q_toked와 a_toked를 이어붙인 것임
        학습에 사용되는 정답 문장은 a_toked임
        '''

        # 질문의 길이가 최대길이(max_len=120)보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        '''
        q_len > 120         47개
        q_len+a_len > 120   52개

        q_len > 150         1개
        q_len+a_len > 150   8개

        길면 문장 뒤에서부터 max_len의 반만큼을 잘라서 사용
        '''

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


In [ ]:
# ChatbotDataset 객체 생성
# train_data = ChatbotDataset(train_df, max_len=150)
# valid_data = ChatbotDataset(valid_df, max_len=150)

# ChatbotDataset 데이터를 딕셔너리 형태로 변환
def datasetToDatadict(dataset):
    data_dict = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
    }

    for i in range(len(dataset)):
        token_ids, mask, labels_ids = dataset[i]
        data_dict["input_ids"].append(token_ids)
        data_dict["attention_mask"].append(mask)
        data_dict["labels"].append(labels_ids)

    return Dataset.from_dict(data_dict)


# DatasetDict에 넣을 dict 반환
def dataDictToDict(train_data, valid_data):
    train = datasetToDatadict(train_data)
    validation = datasetToDatadict(valid_data)

    return {"train": train, "validation": validation}


# 딕셔너리 데이터를 datasets.Dataset 형태로 변환
# DatasetDict = dataDictToDict(train_data, valid_data)
# DatasetDict

In [ ]:
def create_new_dir(base_dir):
    # 디렉토리가 이미 존재하는지 확인
    if os.path.exists(base_dir):
        # 맨 뒤에 붙은 숫자를 찾기 위한 정규 표현식
        match = re.search(r'\d+$', base_dir)

        if match:
            # 맨 뒤에 붙은 숫자를 1 증가
            new_dir = base_dir[:match.start()] + str(int(match.group()) + 1)
        else:
            # 맨 뒤에 숫자가 없으면 '_1'을 붙임
            new_dir = base_dir + '_1'
    else:
        # 디렉토리가 존재하지 않으면 그대로 반환
        new_dir = base_dir

    # 새로운 디렉토리 생성
    os.makedirs(new_dir, exist_ok=True)

    return new_dir


# Trainer Parameter 설정 및 Train

In [ ]:
new_dir = create_new_dir(f'./{date}_1')
print(new_dir)
# 모델 학습을 위한 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir=f'{new_dir}/results',  # 모델 결과물 저장 디렉토리
    logging_dir=f'{new_dir}/logs',  # 학습 로그 저장 디렉토리
    num_train_epochs=EPOCHS,  # 학습 에폭 수
    per_device_train_batch_size=TRAIN_BATCH_SIZE,  # 각 디바이스당 학습 배치 크기
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,  # 가중치 감소 (Weight decay) 설정
    learning_rate=LR,  # 학습률 설정
    logging_steps=100,  # 학습 로그를 기록하는 간격 (매 100번의 배치마다 로그를 기록)
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2
)

./2024-02-13_1


In [ ]:
class MyModelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        token_ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = inputs['labels']
        outputs = model(token_ids)
        logits = outputs.logits
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=logits.shape[2], dim=2)
        Sneg = -1e18    # Sneg 값은 적절하게 설정해주세요.
        mask_out = torch.where(mask_3d == 1, logits, Sneg * torch.ones_like(logits))
        criterion = torch.nn.CrossEntropyLoss()  # 손실 함수는 적절히 선택해주세요.
        loss = criterion(mask_out.transpose(2, 1), label)
        avg_loss = loss.sum() / inputs["attention_mask"].sum()

        return (avg_loss, outputs) if return_outputs else avg_loss

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    # 정확도랑 f1까지 나옴


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=koGPT2_TOKENIZER)
new_df_vali = new_df.reset_index(drop=True)
full_data = ChatbotDataset(new_df, max_len=300)
vali_data = ChatbotDataset(new_df_vali.iloc[:,:10], max_len=300)
# 딕셔너리 데이터를 datasets.Dataset 형태로 변환
full_DatasetDict = dataDictToDict(full_data, vali_data)

In [ ]:
full_trainer = MyModelTrainer(
    model,
    training_args,
    train_dataset=full_DatasetDict['train'],
    eval_dataset=full_DatasetDict['validation'],
    tokenizer=koGPT2_TOKENIZER,
    # compute_metrics=compute_metrics
)
full_trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch,Training Loss,Validation Loss
1,0.247900,0.272532
2,0.259400,0.272284
3,0.257400,0.272123
4,0.245900,0.272028
5,0.261700,0.271964
6,0.256900,0.271915
7,0.253000,0.271898
8,0.260800,0.271881
9,0.262800,0.271869
10,0.255000,0.271868


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=12800, training_loss=0.2562604185938835, metrics={'train_runtime': 8054.6111, 'train_samples_per_second': 6.352, 'train_steps_per_second': 1.589, 'total_flos': 7832636928000000.0, 'train_loss': 0.2562604185938835, 'epoch': 20.0})

# 모델 저장

In [ ]:
# 최종 모델 저장할 경로 설정
MODEL_SAVE_PATH = f'{"/content/drive/MyDrive/KDT-챗봇 프로젝트/모델실험실/정흔/최종데이터모델/skt-kogpt2-base-v2"}-{date}_3-{device}.pth'

In [ ]:
torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': full_trainer.optimizer.state_dict()}, MODEL_SAVE_PATH)